# Credit Risk Classification

Credit risk poses a classification problem that’s inherently imbalanced. This is because healthy loans easily outnumber risky loans. In this Challenge, you’ll use various techniques to train and evaluate models with imbalanced classes. You’ll use a dataset of historical lending activity from a peer-to-peer lending services company to build a model that can identify the creditworthiness of borrowers.

## Instructions:

This challenge consists of the following subsections:

* Split the Data into Training and Testing Sets

* Create a Logistic Regression Model with the Original Data

* Predict a Logistic Regression Model with Resampled Training Data 

### Split the Data into Training and Testing Sets

Open the starter code notebook and then use it to complete the following steps.

1. Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

2. Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

    > **Note** A value of `0` in the “loan_status” column means that the loan is healthy. A value of `1` means that the loan has a high risk of defaulting.  

3. Check the balance of the labels variable (`y`) by using the `value_counts` function.

4. Split the data into training and testing datasets by using `train_test_split`.

### Create a Logistic Regression Model with the Original Data

Employ your knowledge of logistic regression to complete the following steps:

1. Fit a logistic regression model by using the training data (`X_train` and `y_train`).

2. Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

3. Evaluate the model’s performance by doing the following:

    * Calculate the accuracy score of the model.

    * Generate a confusion matrix.

    * Print the classification report.

4. Answer the following question: How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

### Predict a Logistic Regression Model with Resampled Training Data

Did you notice the small number of high-risk loan labels? Perhaps, a model that uses resampled data will perform better. You’ll thus resample the training data and then reevaluate the model. Specifically, you’ll use `RandomOverSampler`.

To do so, complete the following steps:

1. Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

2. Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

3. Evaluate the model’s performance by doing the following:

    * Calculate the accuracy score of the model.

    * Generate a confusion matrix.

    * Print the classification report.
    
4. Answer the following question: How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

### Write a Credit Risk Analysis Report

For this section, you’ll write a brief report that includes a summary and an analysis of the performance of both machine learning models that you used in this challenge. You should write this report as the `README.md` file included in your GitHub repository.

Structure your report by using the report template that `Starter_Code.zip` includes, and make sure that it contains the following:

1. An overview of the analysis: Explain the purpose of this analysis.


2. The results: Using bulleted lists, describe the balanced accuracy scores and the precision and recall scores of both machine learning models.

3. A summary: Summarize the results from the machine learning models. Compare the two versions of the dataset predictions. Include your recommendation for the model to use, if any, on the original vs. the resampled data. If you don’t recommend either model, justify your reasoning.

In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv(
    Path('./Resources/lending_data.csv')
)
#index_col="Date",
#parse_dates=True, 
#infer_datetime_format=True
#)
# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y_targets = lending_data_df['loan_status']

# Separate the X variable, the features

x_features = lending_data_df.drop(columns='loan_status')


In [4]:
# Review the y variable Series
y_targets

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [5]:
# Review the X variable DataFrame
x_features

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y_targets.value_counts

<bound method IndexOpsMixin.value_counts of 0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64>

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_training_features, x_testing_features, y_training_targets, y_testing_targets = train_test_split(x_features, y_targets)
x_training_features.shape

(58152, 7)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)


In [9]:
# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(x_training_features)
X_train_scaled = X_scaler.transform(x_training_features)
X_test_scaled = X_scaler.transform(x_testing_features)

In [10]:
# Fit the model using training data
logistic_regression_model.fit(X_train_scaled, y_training_targets)

LogisticRegression(random_state=1)

In [11]:
# Make a prediction using the testing data
training_predictions = logistic_regression_model.predict(X_train_scaled)

#Review predictions
training_predictions


array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [12]:
# Convert those predictions (and actual values) to a DataFrame
results_df = pd.DataFrame({"Prediction Training": training_predictions, "Actual Training": y_training_targets}).reset_index(drop=True)
results_df

,Prediction Training,Actual Training
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
...,...,...
58147,0,0
58148,0,0
58149,0,0
58150,0,0


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [13]:
# Fit the model using training data
logistic_regression_model.fit(X_test_scaled, y_testing_targets)


LogisticRegression(random_state=1)

In [14]:
# Apply the fitted model to the test dataset
testing_predictions = logistic_regression_model.predict(x_testing_features)

# Save both the test predictions and actual test values to a DataFrame
results_df = pd.DataFrame({"Testing Predictions": testing_predictions, " Testing Actual": y_testing_targets})
results_df

,Testing Predictions,Testing Actual
51467,0,0
52623,0,0
9243,0,0
18829,0,0
61935,0,0
...,...,...
65075,0,0
69466,0,0
76463,0,1
3599,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [15]:
# Import the accuracy_score function
# Print the balanced_accuracy score of the model
accuracy_score(y_testing_targets, testing_predictions)

0.9698204704911267

In [24]:
# Generate a confusion matrix for the model

testing_matrix = confusion_matrix(y_testing_targets, testing_predictions)
print(testing_matrix)


[[18799     0]
 [  585     0]]


In [17]:
testing_report = classification_report_imbalanced(y_testing_targets, testing_predictions)
# Then view its output
print(testing_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      1.00      0.00      0.98      0.00      0.00     18799
          1       0.00      0.00      1.00      0.00      0.00      0.00       585

avg / total       0.94      0.97      0.03      0.95      0.00      0.00     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model prediction is with a precision of 100% and 99% for recall using the actual data. The predicted data has an a precision of 86% and recall accuracy of 91%

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [18]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)


# Fit the original training data to the random_oversampler model
y_train = y_training_targets 
X_train = x_training_features

X_resampled, y_resampled = random_oversampler.fit_resample(X_train, y_train)

In [19]:
# Count the distinct values of the resampled labels data

y_resampled.value_counts()

0    56237
1    56237
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [20]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
model.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
X_test = x_testing_features
y_test = y_testing_targets

y_pred = model.predict(X_test)


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [21]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, y_pred)

0.9938682863200126

In [22]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, y_pred))

[[18697   102]
 [    4   581]]


In [23]:
# Print the classification report for the model
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.99      1.00      0.99      0.99     18799
          1       0.85      0.99      0.99      0.92      0.99      0.99       585

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Based on the accuracy shown above, the logistic regression model fit very well with the oversampled to provide and accurate classification with very high degree of precision and recall for healthy loan and high risk loans.
for low risk loan '0' the accuracy precision is 100% and recall 99% and for high risk loan'1' the precision is 85% and the recall accuracy of 100%. 
Comparing the logistics regression data and the resampled data we can see that the precision for the predicted data has droped by 0.01 while the recall ibcreased from 91 to 100%.